In [1]:
import torch
import random
from seeker import White_seeker, Random_select_seeker, Random_gen_seeker, Black_seeker
from utils import Unfair_metric, load_model, get_L_matrix
from data import adult
from train_dnn import get_data
from dnn_models.model import MLP
from distances.normalized_mahalanobis_distances import SquaredEuclideanDistance, ProtectedSEDistances
from distances.binary_distances import BinaryDistance
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
rand_seed = 0
use_protected_attr = True

dataset, train_dl, test_dl = get_data(adult, rand_seed)
dataset.use_protected_attr = use_protected_attr
in_dim = dataset.dim_feature()
out_dim = 2

model = MLP(in_dim, out_dim)
load_model(model, 'MLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, id=rand_seed)

In [3]:
# prepare data
all_X, all_y = dataset.get_all_data(), dataset.labels
all_pred = model.get_prediction(all_X)

In [4]:
all_X.shape

torch.Size([45222, 41])

In [5]:
dataset.dim_feature()

41

In [6]:
adult_gen = adult.Adult_gen(include_protected_feature=use_protected_attr)

In [7]:
# prepare distances
distance_x_NSE = SquaredEuclideanDistance()
distance_x_Causal = ProtectedSEDistances()
distance_y = BinaryDistance()

distance_x_NSE.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen)
distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen, protected_idx=dataset.protected_idxs)

In [8]:
def rand_gen():
    return {
        'age': random.randint(15, 60),
        'capital_gain': 0,
        'capital_loss': 0,
        'education_num': random.randint(1, 15),
        'hours_per_week': random.randint(10, 50),
        'race_white': random.choice([0, 1]),
        'sex_male': random.choice([0, 1]),
        'marital_status': random.choice(list(range(7))),
        'occupation': random.choice(list(range(14))),
        'relationship': random.choice(list(range(6))),
        'workclass': random.choice(list(range(7)))
    }

def perturb_pair(x, pert_features, pert_func):
    pair = dict()
    for k, v in x.items():
        pair[k] = torch.tensor([v, v])
        if k in pert_features:
            pair[k][1] = pert_func(pair[k][0])
    return pair

x = rand_gen()
pair = adult.generate_from_origin(**perturb_pair(x, ['sex_male', 'race_white'], lambda x:1-x))
adult.get_original_feature(pair)

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,0.0,0.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0
1,38.0,0.0,0.0,12.0,27.0,1.0,0.0,4.0,11.0,0.0,1.0


In [18]:
distance_x_Causal(pair[0], pair[1])

tensor([[2.0000e-05]])

In [9]:
print(1 / distance_x_Causal(pair[0], pair[1]))

tensor([[50000.]])


In [10]:
# x = rand_gen()
# for key in x:
#     pair = adult.generate_from_origin(**perturb_pair(x, key, x[key]+1))
#     print(distance_x_NSE(pair[0], pair[1]), 1 / distance_x_NSE(pair[0], pair[1]))

In [11]:
# epsilon越大，要求不公平样本对dx越小，越严格
epsilon = 49999
unfair_metric = Unfair_metric(dx=distance_x_Causal, dy=distance_y, epsilon=epsilon)

In [20]:
L = get_L_matrix(all_X, all_pred, distance_x_Causal, distance_y)
n_unfair = torch.sum(L>epsilon).item()

total_pairs = all_X.shape[0]**2

unfair_ratio = n_unfair/total_pairs
unfair_ratio

  0%|          | 0/91 [00:00<?, ?it/s]

1.858164100499991e-08

In [21]:
1/unfair_ratio

53816560.10526315

In [22]:
p_success = lambda n: 1 - (1 - unfair_ratio)**n
for i in range(10):
    if p_success(10**i) > 1/100:
        print(10**i, p_success(10**i))

1000000 0.018410066836367256
10000000 0.16957396255960455
100000000 0.844041310489222
1000000000 0.9999999914867425


In [15]:
def show_result(result):
    pair, n_query = result
    if pair != None:
        display(adult.get_original_feature(pair), n_query)
    else:
        print('not found')

In [25]:
random.seed(422)
torch.manual_seed(422)

select_seeker = Random_select_seeker(model=model, unfair_metric=unfair_metric, data=all_X)
for _ in range(3):
    show_result(select_seeker.seek(dx_constraint=True, max_query=1e6))

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,53.0,0.0,0.0,9.0,40.0,0.0,1.0,2.0,10.0,0.0,2.0
1,53.0,0.0,0.0,9.0,40.0,1.0,1.0,2.0,10.0,0.0,2.0


7362

10000


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,51.0,0.0,0.0,9.0,50.0,0.0,1.0,2.0,11.0,0.0,2.0
1,51.0,0.0,0.0,9.0,50.0,1.0,1.0,2.0,11.0,0.0,2.0


10386

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,43.0,0.0,0.0,10.0,40.0,0.0,1.0,2.0,11.0,0.0,2.0
1,43.0,0.0,0.0,10.0,40.0,1.0,1.0,2.0,11.0,0.0,2.0


3588

In [23]:
random.seed(422)
torch.manual_seed(422)

gen_seeker = Random_gen_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
for _ in range(3):
    show_result(gen_seeker.seek(by_range=True, max_query=len(all_X)))

KeyboardInterrupt: 

In [ ]:
random.seed(422)
torch.manual_seed(422)

for _ in range(3):
    show_result(gen_seeker.seek(by_range=False, max_query=len(all_X)))

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,55.0,0.0,0.0,12.0,40.0,0.0,0.0,0.0,3.0,4.0,0.0
1,49.0,0.0,0.0,12.0,40.0,0.0,0.0,0.0,3.0,4.0,0.0


26

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,46.0,0.0,0.0,16.0,40.0,1.0,0.0,4.0,11.0,0.0,0.0
1,39.0,0.0,0.0,16.0,40.0,1.0,0.0,4.0,11.0,0.0,0.0


6

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,48.0,0.0,0.0,13.0,40.0,1.0,0.0,2.0,0.0,0.0,2.0
1,36.0,0.0,0.0,13.0,39.0,1.0,0.0,2.0,0.0,0.0,2.0


84

In [ ]:
adult_gen.get_range('data')

tensor([[1.7000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.0000e+01, 9.9999e+04, 4.3560e+03, 1.6000e+01, 9.9000e+01, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000

In [ ]:
random.seed(422)
torch.manual_seed(422)

white_seeker = White_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(white_seeker.seek())
for _ in range(3):
    show_result(white_seeker.seek(max_query=len(all_X)))

1000000000000000.0
x0 changes
1000000000000000.0
x0 changes
819200.0
x0 changes
819200.0
x0 changes
819200.0
x0 changes
819200.0
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,17.0,0.0,2198.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0
1,17.0,0.0,1658.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0


14

1000000000000000.0
x0 changes
1000000000000000.0
x0 changes
819200.0
x0 changes
819200.0
x0 changes
819200.0
x0 changes
819200.0
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,17.0,0.0,2198.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0
1,17.0,0.0,1658.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0


14

1000000000000000.0
x0 changes
1000000000000000.0
x0 changes
819200.0
x0 changes
819200.0
x0 changes
819200.0
x0 changes
819200.0
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,17.0,0.0,2198.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0
1,17.0,0.0,1658.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0


14

In [ ]:
random.seed(422)
torch.manual_seed(422)

black_seeker = Black_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
show_result(black_seeker.seek(max_query=len(all_X)))

0
1
2
3
4
5
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,17.0,0.0,2198.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0
1,17.0,0.0,1659.0,1.0,1.0,0.0,0.0,4.0,7.0,0.0,3.0


476